In [1]:
from clamv import *

import cv2
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X, y = get_images()

# EigenFaces

In [ ]:
test(OpenCVClassifier(cv2.face.createEigenFaceRecognizer), X, y)

(0.58656924449916814, 0.056801596825888756)

In [3]:
test(TransformerClassifier(PCA(), LinearSVC()), X, y)

(0.57698536797657274, 0.040295840076542376)

In [4]:
pca = PCA(n_components=150, svd_solver='randomized', whiten=True)
param_grid = {
    'C': [1e3, 5e3, 1e4, 5e4, 1e5],
    'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), 
                   param_grid)
test(TransformerClassifier(pca, clf), X, y)

(0.85479823228778895, 0.035829553692219233)

# FisherFaces

In [4]:
test(OpenCVClassifier(cv2.face.createFisherFaceRecognizer), X, y)

(0.71804585236085305, 0.039591290965637224)

In [6]:
test(TransformerClassifier(LinearDiscriminantAnalysis(), LinearSVC()),
     X, y)

/usr/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.


(0.76468663961130423, 0.034843890262949616)